In [87]:
import codecs
from collections import defaultdict

definitions = defaultdict(list)
with codecs.open(r'locale\ru\vocab\ozhegov.txt', 'r', 'utf8') as f:
    description = f.readline().split('|')
    states = [line.split('|') for line in f]
    for s in states:
        definitions[s[0]].append(s)

In [6]:
print (description)

['VOCAB', 'BASEFORM', 'PHONGL', 'GRCLASSGL', 'STYLGL', 'DEF', 'ANTI', 'LEGLEXAM\r\n']


In [88]:
for states in definitions.values():
    for i in range(1,len(states)):
        if states[i-1][4].startswith('!'):
            atakze = states[i][4].rfind(u'а также')
            s4 = states[i][4][atakze:] if atakze > 0 else states[i][4]
            states[i][4] =states[i-1][4][1:]+' '+ states[i-1][5]+', '+ s4

In [70]:
u'лесное травянистое растение сем. розоцветных со с¦едобными ярко-красными кислыми ягодами, а также'.split(u'а также')[-1]

''

In [89]:
import re

abbrs = {r'\bobs\.?': u'устаревшее',
        r'\bустар\.': u'устаревшее',
        r'\bстар.': u'в старину',
        r'\bColloq\.?': u'разговорное',
        r'\bразг\.': u'разговорное',
        r'\bшутл\.': u'шутлилое',
        r'\bофиц\.': u'официальное',
        r'\bвысок\.': u'высокое',
        r'\bупотр\.': u'употребляется',
        r'\bобл\.': u'местное',
        r'\bпрост\.': u'просторечное',
        r'-н\.': u'-нибудь',
        r'-л\.': u'-либо',
        r'\bнеодобр\.': u'неодобрительное',
        r'\bnon-st\.': u'',
        r'\bсо знач\.': u'со значением',
        r'\bперен\.': u'переносное',
        r'\bв знач\.': u'в значении',
        r'\bSpec': u'',
        r'\bLib': u'',
        r'\bирон\.': u'ироничное',
        r'\bспец\.': u'специальное',
        r'\bкнижн\.': u'книжное',
       }

for states in definitions.values():
    for s in states:
        for (abbr,value) in abbrs.items():
            for i in [4,5]:
                s[i] = re.sub(abbr, value, s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'\(-\w+\)', '', s[i])  #изобразить (-ать)
                s[i] = re.sub(r'==', u'то же, что и ', s[i])
                s[i] = re.sub(r'<=', 'от слова ', s[i])
                s[i] = re.sub(r'\d*\s*N\d*\W*\d*','', s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'[^а-яё0-9\",\.:\(\)\s-]+', '', s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'\(\s+', '(', s[i])
                s[i] = re.sub(r'\s+\)', ')', s[i])
                s[i] = re.sub(r'\s+', ' ', s[i])

In [92]:
with codecs.open(r'locale\ru\vocab\ozhegov.hint.txt', 'w+', 'utf8') as f:
    for word in sorted(definitions.keys()):
        for s in definitions[word]:
            worddef = (s[4]+' '+s[5]).strip()
            if worddef != "":
                f.write(s[0]+'|'+worddef+'\n')
        